In [ ]:
from pandas import DataFrame, read_csv
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from numpy import transpose
import numpy as np
from math import sqrt
import tensorflow as tf
from tensorflow.keras.layers import Dense, Activation, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import SGD,RMSprop,Adam
from time import time
import seaborn as sns
#tf.debugging.set_log_device_placement(True)
print(tf.__version__)

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

#a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
#b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
#c = tf.matmul(a, b)

#print(c)

In [ ]:
X_train_df = read_csv('X_train.csv', index_col=0)
X_val_df = read_csv('X_val.csv', index_col=0)
test_df = read_csv('X_test.csv', index_col=0)
y_train_df = read_csv('y_train.csv', index_col=0)
y_val_df = read_csv('y_val.csv', index_col=0)

In [ ]:
y_train_df.shape

### Output scaling

In [ ]:
scaler_Y = StandardScaler()
scaler_Y.fit(y_train_df)

y_train = scaler_Y.transform(y_train_df)
y_val = scaler_Y.transform(y_val_df)

In [ ]:
y_train = y_train.SalePrice.values.astype(float)
y_val = y_val.SalePrice.values.astype(float)

In [ ]:
X_train = transpose([X_train.Surface.values.astype(float)])
X_val = transpose([X_val.Surface.values.astype(float)])

In [ ]:
X_train_df.shape

In [ ]:
#X_train = X_train[['Surface','OverallQual']].values.astype(float)
#X_val = X_val[['Surface','OverallQual']].values.astype(float)
X_train = X_train_df.values.astype(float)
X_val = X_val_df.values.astype(float)
test = test_df.values.astype(float)

In [ ]:
nb_features = 75 # 24
X_train = X_train[:,:nb_features]
X_val = X_val[:,:nb_features]
test = test[:,:nb_features]

In [ ]:
nb_features = 150
pca = PCA(n_components=nb_features)
pca.fit(X_train)
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('Number of components')
plt.ylabel('Cumulative explained variance')
X_train=pca.transform(X_train)
X_val=pca.transform(X_val)
test=pca.transform(test)


In [ ]:
X_train.shape

In [ ]:
from tensorflow.keras.backend import sigmoid, tanh, maximum
def custom(x):
    return maximum(x,tanh(x))

In [ ]:
def get_reg_model():
    return Sequential([
                    Input(shape=X_train.shape[1]),
                    Dense(1)
])

In [ ]:
def get_other_model():
    return Sequential([
                    Input(shape=X_train.shape[1]),
                    Dense(32, kernel_regularizer=regularizers.l1(0.001)),
                    Activation('tanh', activity_regularizer=regularizers.l1(0.001)),
#                    Dense(32, kernel_regularizer=regularizers.l1(0.001)),
#                    Activation('tanh', activity_regularizer=regularizers.l1(0.001)),
                    Dense(1)
])

In [ ]:
def get_noregu_model():
    return Sequential([
                    Input(shape=X_train.shape[1]),
                    Dense(32),
                    Activation('softplus'),
#                    Dense(32),
#                    Activation('tanh'),
                    Dense(1)
])

In [ ]:
loss = 'mse'
LEARNING_RATE = 0.01

In [ ]:
model = get_reg_model()
model.compile(loss=loss, optimizer=SGD(lr=LEARNING_RATE))
BATCH_SIZE = X_train.shape[0] # computing the loss over the whole dataset
EPOCHS = 750 # how many iterations over the whole dataset
t_0 = time()
#with tf.device('/device:GPU:0'):
history = model.fit(X_train, y_train,  validation_data=(X_val, y_val), epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=0)

print(time()-t_0, 's')

In [ ]:
DataFrame(history.history).plot(figsize=(8, 5), logy=True)
plt.grid(True)
#plt.gca().set_ylim(0, 1)
plt.title('Model performance throughout training')
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
y_p_scaled = model.predict(X_val)
y_p_scaled = y_p_scaled.reshape(y_p_scaled.shape[0])
y_p = scaler_Y.inverse_transform(y_p_scaled)

RMSLE = sqrt(mean_squared_error(y_val_df,y_p))
print('Validation RMSLE:', RMSLE)
y_p_scaled = model.predict(X_train)
y_p_scaled = y_p_scaled.reshape(y_p_scaled.shape[0])

y_p = scaler_Y.inverse_transform(y_p_scaled)

RMSLE = sqrt(mean_squared_error(y_train_df.SalePrice,y_p))
print('Train RMSLE:', RMSLE)

In [ ]:
test_pred_log_scaled = model.predict(test)
test_pred_log = scaler_Y.inverse_transform(test_pred_log_scaled)
test_pred = np.exp(test_pred_log).reshape(test_pred_log.shape[0])
submission = DataFrame({"SalePrice": test_pred}, index=test_df.index)
submission.to_csv('test-prediction-keras-lin.csv')

In [ ]:
model = get_other_model()
model.summary()

In [ ]:
LEARNING_RATE=0.05
model.compile(loss=loss, optimizer=SGD(lr=LEARNING_RATE))
#model.compile(loss=loss, optimizer=RMSprop(lr=LEARNING_RATE))
#model.compile(loss=loss, optimizer=Adam(lr=LEARNING_RATE))

In [ ]:
BATCH_SIZE = X_train.shape[0] # computing the loss over the whole dataset
EPOCHS = 50000 # how many iterations over the whole dataset
t_0 = time()
#with tf.device('/device:GPU:0'):
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=0)
print(time()-t_0)

In [ ]:
DataFrame(history.history).plot(figsize=(8, 5), logy=True)
plt.grid(True)
#plt.gca().set_ylim(0, 1)
plt.title('Model performance throughout training')
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
y_p_scaled = model.predict(X_val)
y_p_scaled = y_p_scaled.reshape(y_p_scaled.shape[0])
y_p_val = scaler_Y.inverse_transform(y_p_scaled)

RMSLE = sqrt(mean_squared_error(y_val_df,y_p_val))
print('Validation RMSLE:', RMSLE)
y_p_scaled = model.predict(X_train)
y_p_scaled = y_p_scaled.reshape(y_p_scaled.shape[0])

y_p_train = scaler_Y.inverse_transform(y_p_scaled)

RMSLE = sqrt(mean_squared_error(y_train_df.SalePrice, y_p_train))
print('Train RMSLE:', RMSLE)

In [ ]:
test_pred_log_scaled = model.predict(test)
test_pred_log = scaler_Y.inverse_transform(test_pred_log_scaled)
test_pred = np.exp(test_pred_log).reshape(test_pred.shape[0])
submission = DataFrame({"SalePrice": test_pred}, index=test_df.index)
submission.to_csv('test-prediction-keras-1hiddenLayer.csv')

In [ ]:
plt.figure(figsize=(15,8))
plt.subplot(1,2,1)
sns.distplot(y_p_train-y_train_df.SalePrice)
plt.subplot(1,2,2)
sns.distplot(y_p_val-y_val_df.SalePrice)
plt.show()

In [ ]:
y = y_val_df.SalePrice.to_numpy()
y = y.reshape(y.shape[0])
print(y_p_train.shape)
df = DataFrame(data={'value':np.exp(y), 'predicted':np.exp(y_p_val)},index=y_val_df.index)
plt.figure(figsize=(15,10))
sns.scatterplot(x='value',y='predicted', data=df )
plt.show()

In [ ]:
anomaly_idx = df['predicted'].idxmax()
print(anomaly_idx)

In [ ]:
anomaly = X_val_df.loc[anomaly_idx]

In [ ]:
anomaly.loc[np.abs(anomaly)>1.5]